In [4]:
import pandas
import glob


In [8]:
files = glob.glob("*.csv")

for x in files:
   if (x[0] == "C"):
    pass
   else:

       df = pandas.read_csv(x)
        
        # Remove all put and call holdings for now
       df= df[df.putCall != 'Call']
       df= df[df.putCall != 'Put']
        
        # Remove the PRN holdings
       df= df[df.sshPrnamtType != 'PRN']
        
        #Combine all duplicate entries
       df = df.groupby(['nameOfIssuer']).sum()
     
        #Remove columns that we don't need
       df = df[df.columns.difference(['Sole', 'Shared','None','titleOfClass','cusip','sshPrnamtType','investmentDiscretion','otherManager','putCall'])]
       
          #
       df['sprice'] = (df['value'] / df['sshPrnamt']) *1000 
       df = df[df.columns.difference(['value'])]
        
       df.to_csv("Clean"+x)
        

In [10]:
#retrieve and sort the cleaned files
cfiles = glob.glob("*.csv")
cfiles = sorted(cfiles)

#make our final dataframe to populate
fdf = pandas.DataFrame()

#Merge all of the data tables together; the names of columns are ugly so we need to rename them
merged = pandas.read_csv(cfiles[4])
for x in range(int(len(cfiles)/2)+1,len(cfiles)):
    merged = pandas.merge(merged, pandas.read_csv(cfiles[x]), on = ['nameOfIssuer','nameOfIssuer'])

#accumulator for initial new column names
colnames = ['Company']

    
for x in range(0,len(cfiles)):
    if ((cfiles[x])[0] == "C"):
        
        #Making the column headers for our new table from our old ones
        amt = (cfiles[x])[5:9] + "price"
        mon = (cfiles[x])[5:9] + "amt"    
        colnames.append(amt)
        colnames.append(mon)

#put logic column names on columns        
merged.columns = colnames

    
for x in range(0,len(cfiles)):
    if ((cfiles[x])[0] == "C"):
        #add columns for calculated difference in number of shares
        if (x < len(cfiles)-1):
        
            first = (cfiles[x])[5:9]
            second = (cfiles[x+1])[5:9]
            diff= first + "-" + second + "chng"
            colnames.append(diff)
        
            merged[diff]= merged[(second+"amt")] - merged[(first+"amt")]
             
        
#add col headers for the differences        
merged.columns = colnames
merged.to_csv("Final.csv")
#merged